<a href="https://colab.research.google.com/github/vijayshankarrealdeal/GoogleColab/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm

In [2]:
!wget --no-check-certificate \
    http://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -o /cats_and_dogs_filtered.zip

In [3]:
import tensorflow as tf
import numpy as np
import os
import zipfile
from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
dataset_path = './cats_and_dogs_filtered.zip'

In [5]:
zip_object = zipfile.ZipFile(dataset_path,mode= 'r')
zip_object.extractall("./")

In [6]:
zip_object.close()

In [7]:
dataset_path_new = "/content/cats_and_dogs_filtered"

In [8]:
train_dir = os.path.join(dataset_path_new,"train")
validation_dir = os.path.join(dataset_path_new,"validation")

In [9]:
IMG_SHAPE = (128,128,3)

In [10]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [11]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [12]:
base_model.trainable = False

In [13]:
globle_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [14]:
prediction_layer = tf.keras.layers.Dense(1,activation="sigmoid")(globle_average_layer)

In [15]:
model = tf.keras.models.Model(inputs = base_model.input,outputs= prediction_layer)

In [16]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss = 'binary_crossentropy',metrics=['accuracy'])

In [17]:
data_gen_train = ImageDataGenerator(
    rescale = 1/255.0,
)
data_gen_valid = ImageDataGenerator(
    rescale = 1/255.0,
)

In [18]:
train_gen = data_gen_train.flow_from_directory(train_dir,target_size=(128,128),batch_size=128,class_mode='binary')

Found 2000 images belonging to 2 classes.


In [19]:
valid_gen = data_gen_valid.flow_from_directory(validation_dir,target_size=(128,128),batch_size=128,class_mode='binary')

Found 1000 images belonging to 2 classes.


In [20]:
model.fit(train_gen,epochs =20,validation_data=valid_gen)

Epoch 1/20
16/16 [==============================] - 43s 579ms/step - loss: 0.7002 - accuracy: 0.5979 - val_loss: 0.5812 - val_accuracy: 0.7200
Epoch 2/20
16/16 [==============================] - 8s 485ms/step - loss: 0.5694 - accuracy: 0.7138 - val_loss: 0.5053 - val_accuracy: 0.7870
Epoch 3/20
16/16 [==============================] - 8s 481ms/step - loss: 0.4832 - accuracy: 0.7887 - val_loss: 0.4445 - val_accuracy: 0.8300
Epoch 4/20
16/16 [==============================] - 7s 472ms/step - loss: 0.4249 - accuracy: 0.8274 - val_loss: 0.3943 - val_accuracy: 0.8620
Epoch 5/20
16/16 [==============================] - 8s 480ms/step - loss: 0.3885 - accuracy: 0.8525 - val_loss: 0.3533 - val_accuracy: 0.8850
Epoch 6/20
16/16 [==============================] - 7s 475ms/step - loss: 0.3409 - accuracy: 0.8872 - val_loss: 0.3174 - val_accuracy: 0.8960
Epoch 7/20
16/16 [==============================] - 7s 474ms/step - loss: 0.3047 - accuracy: 0.8966 - val_loss: 0.2872 - val_accuracy: 0.9140
Epoch

## **Fine Tuning**

In [21]:
base_model.trainable = True

In [22]:
len(base_model.layers)

154

In [23]:
fine_tune_at = 100

In [24]:
for layers in base_model.layers[:fine_tune_at]:
  layers.trainable = False

In [25]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss = 'binary_crossentropy',metrics=['accuracy'])